In [57]:
# test the use of runlength encoding for rainfields rain_rate data 

In [58]:
import pymongo 
from pymongo import MongoClient
from netCDF4 import Dataset 
import collections
import bson

In [59]:
rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
rf3 = Dataset(rf3_name)
stn_id = int(rf3.getncattr("station_id"))

In [60]:
# first check if the projection already exists in the database 
client = MongoClient()
db = client["test"]
collection = db["projection"] 

# get the number of documents for stn_id 
count_docs = collection.count_documents(filter={"stn_id": stn_id})
print(f"Found {count_docs} documents for this projection") 

Found 6 documents for this projection


In [62]:
proj = {}
if count_docs == 0 : 
     
    # get the projection from the ncfile and write it to the database 
    proj_attr = {}
    proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
    standard_parallel = rf3.variables['proj'].getncattr("standard_parallel")
    proj_attr['standard_parallel'] =  standard_parallel.tolist()
    proj_attr['longitude_of_central_meridian'] = rf3.variables['proj'].getncattr("longitude_of_central_meridian")
    proj_attr['latitude_of_projection_origin'] = rf3.variables['proj'].getncattr("latitude_of_projection_origin")
    proj_attr['false_easting'] = rf3.variables['proj'].getncattr("false_easting") 
    proj_attr['false_northing'] = rf3.variables['proj'].getncattr("false_northing") 
    proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
    towgs84 = rf3.variables['proj'].getncattr("towgs84")
    proj_attr['towgs84'] = towgs84.tolist()
    
    x_attr = rf3.variables['x'].__dict__ 
    x = rf3['x'][:].tolist()
    x_attr['x'] = x 
    
    y_attr = rf3.variables['y'].__dict__ 
    y = rf3['y'][:].tolist()
    y_attr['y'] = y 
    
    proj['stn_id'] = int(stn_id)
    proj['proj'] = proj_attr 
    proj['y'] = y_attr
    proj['x'] = x_attr

    proj_id = collection.insert_one(proj).inserted_id
else:
    # get the projection from the database 
    proj = collection.find_one({"stn_id": stn_id}) 
    proj_id = proj['_id']


dict_keys(['_id', 'stn_id', 'proj', 'y', 'x'])
